# Introduction to Discrete Optimization

```{admonition} Learning outcomes
After reading this note, you will be able to:
- Define a discrete optimization problem
- Model and solve discrete optimization problems
```

While useful in contexts as varied as [engineering design](https://flowlab.groups.et.byu.net/mdobook.pdf) and deep learning, continuous optimization cannot model *discrete* decisions. For example, in a vehicle routing application, does this large package go on this delivery truck or the other? In a production planning setting, how many units of products A, B, and C should a company produce to maximize its revenue?

To tackle such problems, we turn to *discrete optimization*. Some things we have learned in the continuous optimization lectures transfer to the discrete setting. For instance, some discrete problems are easier to solve than others. A surprising example of this split is that it is easy to find the *shortest path* between two nodes in a graph with non-negative edge weights (using [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)), but finding the *longest path* (with no repeated vertices) is extremely challenging! "Easy" discrete optimization problems have efficient algorithms that can quickly find an optimal solution. "Hard" discrete optimization problems require some kind of exhaustive search over the space of all possible solutions, of which there are typically **many**.

In this lecture, we will focus on *exact* (or *global*, or *complete*)  algorithms for hard discrete optimization problems. Run on an instance of an optimization problem for a sufficient amount of time, an exact algorithm either finds an optimal solution and *certifies its optimality*, i.e., it eliminates the possibility that there are any strictly better solutions, or proves that the instance is infeasible. A good exact algorithm uses a combination of divide-and-conquer search (i.e., recursively splitting up a problem into smaller subproblems that are hopefully easier to solve than the original problem), heuristics that find feasible solutions, preprocessing rules that reduce the number of decision variables and constraints, and domain reduction techniques to restrict the domains of the discrete variables or the feasible region. 

A sensible combination of these elements into an exact solver typically averts the need to examine all possible variable assignments. This makes it possible to tackle extremely challenging real-world problems every day, for example in the [airline industry](https://sites.rutgers.edu/jian-yang/wp-content/uploads/sites/191/2019/06/handbook.pdf) or at the scale of a [governmental response to COVID-19](https://www.gurobi.com/news/chile-receives-franz-edelman-award-for-covid-19-research-supported-by-gurobi/).

## Bin packing, continuous and discrete 
```{image} https://www.scipopt.org/doc/html/binpacking.png 
:label: binpacking
:alt: Bin Packing instance
:align: center

A bin packing instance courtesy of the [SCIP documentation](https://www.scipopt.org/doc/html/BINPACKING_PROBLEM.php)
```

We will build intuition using two versions of the a fundamental problem in discrete optimization: bin packing. You are given $n$ cups of liquids each of size $w_j>0$ and as many buckets of size $c>0$. The liquid packing problem asks you to choose the fraction of liquid $i$ to empty into bucket $j$ such that no bucket overflows, i.e., you do not pour more than $c$ liters in any bucket and you minimize the number of buckets used. A bucket is used if its corresponding binary variable $y_i$ is equal to one. The fraction of liquid $i$ to empty into bucket $j$ is represented by the **continuous** variable $x_{ij}\in[0,1]$.

```{math}
\min_{\boldsymbol{x}, \boldsymbol{y}} \quad & \sum_{i=1}^{n}{y_i}\\
\text{subject to} \quad 
& \sum_{j=1}^{n}{w_j x_{ij}} \leq c y_i, \quad &\forall i\in[n],\\
& \sum_{i=1}^{n}{ x_{ij}} = 1, \quad&\forall j\in[n],\\
& y_i\in\{0,1\}, \quad &\forall i\in[n],\\
& x_{ij}\in[0,1], \quad &\forall i\in[n],j\in[n].
```

An optimal solution to this problem can be obtained in the intuitive way: in any order, pour the liquids one after the other into a bucket until it is full. Repeat the same procedure using a new bucket and stop when all liquids have been poured. The optimal value of the solution $(\boldsymbol{x}^\star, \boldsymbol{y}^\star)$ produced by this algorithm is:
```{math}
\sum_{i=1}^{n}{y^{\star}_i}=\Bigg\lceil{\frac{1}{c}\sum_{i=1}^{n}{{w_j}}}\Bigg\rceil.
```
Note that the liquid packing problem has a mix of discrete and continuous variables! However, the binary $\boldsymbol{y}$ variables are implied by the continuous $\boldsymbol{x}$ variables and are thus secondary. This problem admits an efficient algorithm as we've just argued. Let's make things more interesting.

Instead of packing liquids, we will now pack rectangular boxes. They all have the same width as the bins they will be packed into, but their heights $w_j>0$ vary. The formulation is the same with a crucial difference: boxes cannot be split into multiple bins! As such, we replace the last set of constraints with $x_{ij}\in\{0,1\}, \forall i\in[n],j\in[n]$. This seemingly minute change makes the bin packing problem theoretically intractable.

### Enumeration?
A simple, parallelizable, algorithm for bin packing is *exhaustive enumeration*. Let's count how many possible bin arrangements there are for $n$ items and $b$ bins; we assume $b\leq n$ as in most cases one will not need to use $n$ bins as more than one item will be packed into most bins. Of course, not all packings of the items into $b$ bins is feasible in the capacity constraint. However, an upper bound is given by the Stirling number of the second kind, a formula for the number of partitions of a set of $n$ items into $b$ subsets. For a bin packing instance with say $n=50$ items and $b=10$ bins, this number is [$\approx 2.6\times 10^{43}$](https://www.wolframalpha.com/input?i=StirlingS2%5B50%2C10%5D)! Given that bin packing models logistics problems where containers need to be packed with three-dimensional rectangular boxes, for example in [this paper by researchers at Alibaba](https://arxiv.org/abs/1804.06896), we cannot rely strictly on enumeration. Even with a supercomputer that checks each of these possible assignments in parallel, enumeration will take a grotesque amount of time.

### Problem reduction 
The first thing we will do is to try to reduce the number of decisions we need to make. For example, if an instance of the problem has bin capacity $c=10$,   the smallest item has size $2$, and the largest item has size $9$, then the first bin can be assigned to the latter and closed to additional items. One need only search for ways to pack the remaining items into additional bins. In other words, any optimal solution to this instance includes a bin with only the size-$9$ item, so we can fix $x^{\star}_{1i}=1, y^{\star}_1=1$, where $i$ is the index of said item. More generally, any valid preprocessing algorithm that seeks to reduce the problem instance must preserve optimality and feasibility (or infeasibility).

This idea can be generalized to more than one item. For an item $i$, if there is a set $S$ of $k-1$ other items which fit together in a bin with $i$ such that:
- There does not exist another set $S'$ which is larger than $S$ and that also fits with $i$, or;
- Any other set $S'$ fills has total size less than that of $S$.
Clearly, checking all such possible sets $S$ is intractable. However, one could check for small sets with size for example at most 3. We will not discuss the algorithm that does this here, but provide an example of the reduction next.

Let $n=12$ and $c=100$. The items have sizes
```{math}
[99, 93, 90, 88, 80, 10, 10, 6, 5, 5, 4, 4]
```
1. 99 must be in its own bin; 93 can be packed with at most one more item and the best item is 6 (largest that fits) since it minimizes wasted space.
2. Reduced instance: $[90, 88, 80, 10, 10, 5, 5, 4, 4]$. 90 can be packed with at most two other items. To minimize wasted space, we should use either 10 or 5+5. 10 is a better choice since the two 5s can be potentially put split into separate bins later on whereas 10 cannot.
3. Reduced instance: $[88, 80, 10, 5, 5, 4, 4]$. 88 can be packed with at most two other items. 10 dominates 5+5 for the same reason as in step 2.
4. Reduced instance: $[80, 5, 5, 4, 4]$. The sizes sum up to $98\leq c$ so we put them in one bin.

The reduction algorithm optimally solves this instance. This will not be the case in general, but reducing the number of decisions we need to make is always a good thing.

### Feasible solutions using heuristics
Let's look at another instance with $n=12$ and $c=100$. The items have sizes
```{math}
[50, 3, 48, 53, 53, 4, 3, 41, 23, 20, 52, 49]
```
It may be possible to reduce the instance but let us ignore that for a minute. We would like to construct a good solution using a fast heuristic algorithm. A *heuristic* is an algorithm for which there are no theoretical performance guarantees on the problem that it addresses. An *approximation* algorithm is one for which worst-case performance guarantees can be derived. In this lecture, we will not expand on approximation algorithms and how their guarantees are derived; chapter 11 of the book [*Algorithm design* by Kleinberg and Tardos](https://librarysearch.library.utoronto.ca/permalink/01UTORONTO_INST/14bjeso/alma991106508891106196) is a good introduction to this rich topic. We will nonetheless describe and use some approximation algorithms for bin packing.

In the previous example of problem reduction, the items were sorted in decreasing order of their weights. This is a order in which to process the items because larger ones need to be assigned early on to bins whereas smaller ones may be more flexibly appended to bins which have a small amount of remaining space. Sorting our current items yields
```{math}
[53, 53, 52, 50, 49, 48, 41, 23, 20, 4, 3, 3].
```
The two algorithms have "Decreasing" in their name because they process the items one by one in decreasing order of their weights.
1. **First-Fit Decreasing** (FFD) packs the net item into the lowest indexed bin where it fits, or opens a new bin if it does not fit in any of the existing ones.
2. **Best-Fit Decreasing** (BFD) packs the net item into a bin where it fits and minimizes residual space, or opens a new bin if it does not fit in any of the existing ones.

Let's see how FFD and BFD work on our example. They make the same choices in the first 9 iterations:
1. The largest four items need their separate bins so the first 4 iterations give $\{53\}, \{53\},\{52\},\{50\}$.
2. Item $49$ fits best and first with $50$, $48$ with $52$, $41$ with the first bin that has $53$, $23$ with the second bin that has $53$. Eight iterations in, the current packing for both FFD and BFD is $\{53,41\}, \{53,23\},\{52,48\},\{50,49\}$.
3. Item $20$ fits best and first in the second bin, giving: $\{53,41\}, \{53,23,20\},\{52,48\},\{50,49\}$.
4. Item $4$ is where FFD and BFD diverge. The first bin can fit $4$, leading to residual space of $100-(53+41+4)=3$; this is FFD's choice. As for BFD, it observes that the second bin can also fit $4$ but with smaller residual space, $100-(53+23+20+4)=0$.\
   FFD:$\{53,41,4\}, \{53,23,20\},\{52,48\},\{50,49\}$\
   BFD:$\{53,41\}, \{53,23,20,4\},\{52,48\},\{50,49\}$
5. The last two items are of size $3$. BFD can fit both of them in the first bin. FFD can fit only one of them in the second bin and has to open a new fifth bin for the other $3$. The final solution are then:
```{math}
\begin{align}
   \text{FFD}:\; &\{53,41,4\}, &\{53,23,20,3\},\; &\{52,48\},&\{50,49\}, \;&\{3\},\\
   \text{BFD}:\; &\{53,41,3,3\}, &\{53,23,20,4\},\; &\{52,48\},&\{50,49\}\;.
\end{align}
```
The optimal packing for this instance uses $4$ bins, so BFD is optimal whereas FFD is not in this case. In general, heuristics/approximations may not find an optimal solution. Even when they do, we need to computationally *prove* the optimality of a feasible solution, i.e., we cannot guarantee that a solution is optimal until we have eliminated the possibility that there are other better solutions. The next section is a step towards providing such a computational guarantee.

### Lower bounds using relaxations
A *relaxation* of a set $\Omega\subset\mathbb{R}^n$ is a possibly larger set $\Omega'$ that subsumes $\Omega$, i.e., $\Omega\subseteq\Omega'$. In a discrete optimization problem, a simple way to relax is by dropping some or all of the integrality constraints. For bin packing, we have already looked at one such relaxation when we packed liquids instead of solid boxes! To do so, we let the $x_{ij}\in[0,1]$ rather than $x_{ij}\in\{0,1\}$; clearly, $[0,1]$ is a subset of $\{0,1\}$. Relaxing the complicating constraints of bin packing, namely that items cannot be split across bins, allows for an easier optimization problem.

Why care about relaxations if they are in some sense a simplification of the original problem we are interested in solving? Let $z^{\star}(\Omega)$ be the *optimal value* of the problem $\min_{\boldsymbol{x}\in\Omega}f(\boldsymbol{x})$. The following inequality holds true for a relaxation $\Omega'$ of $\Omega$:
```{math}
z^{\star}(\Omega')\leq z^{\star}(\Omega).
```
In words, the optimal value of the relaxation *lower bounds* that of the original problem. It is easy to see why: all optimal solutions in $\Omega$ are also in the superset $\Omega'$. Since $\Omega'$ potentially includes additional optima that are not in $\Omega$, its optimal value with respect to objective function $f$ may be better (lower).

Let's relax bin packing using the liquid version:
```{math}
\Omega=\left\{
x_{ij}\in\{0,1\}  \;\forall i\in[n],j\in[n], y_i\in\{0,1\} \;\forall i\in[n],j\in[n] \; \Bigg| \;
\sum_{j=1}^{n}{w_j x_{ij}} \leq c y_i, \; \forall i\in[n],
\sum_{i=1}^{n}{ x_{ij}} = 1, \;\forall j\in[n].
\right\}\\
\Omega'=\left\{
x_{ij}\in[0,1], \;\forall i\in[n],j\in[n], y_i\in\{0,1\} \;\forall i\in[n],j\in[n] \; \Bigg| \;
\sum_{j=1}^{n}{w_j x_{ij}} \leq c y_i, \; \forall i\in[n],
\sum_{i=1}^{n}{ x_{ij}} = 1, \;\forall j\in[n].
\right\}
```
We are now ready to use this lower bound. We revisit the example instance from the previous section. BFD found a feasible solution of value $4$. What is the lower bound on this instance's optimal value? We argued when we introduced the liquid packing problem that its optimal value is given by
```{math}
\left\lceil{\frac{1}{c}\sum_{i=1}^{n}{{w_j}}}\right\rceil=\left\lceil{\frac{399}{100}}\right\rceil=4.
```
The lower bound from this relaxation *matches the upper bound* provided by BFD! This proves that BFD's solution is optimal for this instance since its optimal value is also $4$, i.e.,
```{math}
z^{\star}(\Omega')=4\leq z^{\star}(\Omega)\leq 4.
```

Again, we will not always be so lucky and most relaxations to discrete optimization problems do not provide lower bounds that are equal to the optimal value. However, they can help us save time by avoiding suboptimal solutions when incorporated into a larger exact search algorithm, as we'll see shortly.

### A stronger lower bound
Given two relaxations $\Omega'$ and $\Omega''$ of a set $\Omega$, $\Omega'$ is a *stronger relaxation* than $\Omega''$ w.r.t. (with respect to) an objective function $f$ if and only if $z^{\star}(\Omega')\geq z^{\star}(\Omega'')$ *for all instances* of the problem. Clearly, a stronger (or tighter) lower bound is preferable because it is closer to an instance's true optimal value.

Referring to the lower bound from the liquid relaxation as $L_1$, we will look at a stronger bound, $L_2$. Let's illustrate these with an example with $n=9$ and $c=100$:
```{math}
[70, 60, 50, 33, 33, 33, 11, 7, 3].
```
An optimal solution to this instance requires $4$ bins, for example with FFD's solution $\{70,11,7,3\}, \{60,33\},\{50,33\},\{33\}$. However, the liquid bound does not match the optimal value since $L_1=\left\lceil\frac{300}{100}\right\rceil=3$, i.e., it might lead us to believe that there exists a "super"-optimal solution with value $3$, which is not the case. Generally, $L_1$ will provide good lower bounds when item sizes are much smaller than the bin capacity $c$, in which case optimal bins will typically not have much residual capacity left and so splitting an item across bins is a reasonable approximation to the optimum.

Another way to relax bin packing is to simply drop some items and pack the rest! Clearly, this would require at most as many bins as the full problem. This is the first step towards the stronger $L_2$ bound, which discards a few "small" items. What's "small" here? Let $\alpha$ be an integer satisfying $0\leq\alpha\leq\frac{c}{2}$. Items with $w_j<\alpha$ will be ignored. The remaining items will be partitioned into three sets:
```{math}
\begin{align}
J_1&=\{j\in[n]\;|\;w_j > c-\alpha \}\\
J_2&=\{j\in[n]\;|\;c-\alpha \geq w_j > \frac{c}{2} \}\\
J_3&=\{j\in[n]\;|\;\frac{c}{2} \geq w_j \geq\alpha \}
\end{align}
```
Let's apply this partitioning to our example, using $\alpha=33\leq 50$. First, we drop the items with weights $11, 7,$ and $3$. We are left with $[70, 60, 50, 33, 33, 33, 11]$ which are partitioned into:
```{math}
J_1=[70], J_2=[60], J_3=[50, 33, 33, 33, 11].
```
It is generally true for any $\alpha\leq\frac{c}{2}$ that items in $J_1$ and $J_2$ cannot share any bins, meaning that our lower bound is at least $|J_1|+|J_2|$. What about the items in $J_3$? Since they have size at least $\alpha$ each, they cannot be packed in a bin that contains an element from $J_1$, the latter having size $w_j > c-\alpha$, meaning that adding $\alpha$ or more would exceed the capacity $c$. They could fill some of the residual space in the bins of $J_2$ items in addition to new bins. We can use the liquid lower bound to estimate how many bins are needed for $J_3$. Assume you could split items from $J_3$ into multiple bins. Then, the first thing to do is to fill up the residual space in bins of $J_2$ items. The latter currently fill up $|J_2|c-\sum_{j\in J_2}{w_j}$ of their bins; this is just the total capacity of all $J_2$ bins minus their total sizes. Let's fill this space up with as much of the $J_3$ items as we can. The remaining amount to fill requires, by the $L_1$ bound, at least
```{math}
\max\left( 0, \left\lceil \frac{\sum_{j\in J_3}{w_j-(|J_2|c-\sum_{j\in J_2}{w_j})}}{c} \right\rceil  \right)
```
bins. The final $L_2$ bound for a given $\alpha$ is then
```{math}
L_2(\alpha) = |J_1| + |J_2| + \max\left( 0, \left\lceil \frac{\sum_{j\in J_3}{w_j-(|J_2|c-\sum_{j\in J_2}{w_j})}}{c} \right\rceil  \right).
```
Each value of $\alpha$ provides a lower bound, but we prefer stronger ones. As such, the $L_2$ bound tries out all possible values of $0\leq\alpha\leq\frac{c}{2}$ and returns the maximum:
```{math}
L_2 = \max\left\{L_2(\alpha)\;|\; 0\leq\alpha\leq\frac{c}{2}, \alpha\text{ integer}\right\}.
```

Continuing to apply the $L_2$ bound to the previous example, we get
```{math}
L(11) = 0 + 2 + \max(0,\lceil (160-70)/100 \rceil = 3\\
L(33) = 1 + 1 + \max(0,\lceil (149-40)/100 \rceil = 4\\
L(50) = 2 + 0 + \max(0,\lceil (50-0)/100 \rceil = 3,
```
so $L_2=4$ which matches the upper bound from the FFD solution, confirming that it is optimal.

```{admonition} Further reading on bin packing
Many of the examples we've used are from these [lecture slides](https://mathopt.be/Slides_LaRoche_Martello.pdf) by Prof. Silvano Martello, a pioneer in discrete optimization. A complete (possibly outdated) account of the bin packing is given in chapter 8 of the book [Knapsack problems](https://librarysearch.library.utoronto.ca/permalink/01UTORONTO_INST/14bjeso/alma991106724929106196) by Martello and Toth.
```

In [1]:
from sympy import * #Symbol, symbols, sin, cos, Polygon, solve, lambdify, Rational, pi, N
from sympy.plotting import plot
from spb import *
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.rcParams['figure.dpi'] = 200

sns.set_theme()
sns.set_context("notebook", font_scale=1.1, rc={"lines.linewidth": 3.5})
sns.set_style("white")
sns.set_style("ticks")
plt.rcParams.update({
    "text.usetex": True,
    # "font.family": "courier",
    'text.latex.preamble': r'\usepackage{amsfonts}'
})